# Latihan-9b Simple perturbation principles

Latihan ini untuk pemodelan half space atau disebut juga blocky model, atau interface model yang terdiri dari 2-3 block layers digunakan untuk menganalisis respon amplitudo terhadap interface antara top dan bottom layers
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-9b Simple Perturbation.ipynb"_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Program setup dan display sama seperti latihan sebelumnya. Gunakan hasil dari Latihan-3 data _logs_blocked_ untuk latihan selanjutnya
<br> 
        
        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        The bruges library's credits go to contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from matplotlib import cm, pyplot
from scipy.optimize import curve_fit
from scipy.signal import hilbert
from f10kuelapis import *
mpl.style.use('seaborn') # default, seaborn

In [ ]:
disp = np.zeros((60,100), dtype=int) 
disp[0:20,:] = 1
disp[20:40,:] = 2
disp[40:,:] = 3
plt.subplots(figsize=(12,8))
plt.imshow(disp, aspect=1, cmap='Dark2')
# plt.grid(False)
plt.show()

In [ ]:
box = np.zeros((60,10), dtype=int) 
box[0:20,:] = 1
box[20:40,:] = 2
box[40:,:] = 3

In [ ]:
t,wavelet=ricker(20,200,2,0)
print(len(wavelet))
isi=np.array(([3.30, 2.42, 1.85],[3.08, 2.25, 1.72],[3.48, 2.48, 1.85]))

In [ ]:
exec(open('f09dispallstacks.py').read())

In [ ]:
amplmin_in=[syn0[:-1].min(),syn5[:-1].min(),syn10[:-1].min(),syn15[:-1].min(),syn20[:-1].min(),syn25[:-1].min(),
         syn30[:-1].min(),syn35[:-1].min(),syn40[:-1].min(),syn45[:-1].min()]
amplmax_in=[syn0[1:].max(),syn5[1:].max(),syn10[1:].max(),syn15[1:].max(),syn20[1:].max(),syn25[1:].max(),
         syn30[1:].max(),syn35[1:].max(),syn40[1:].max(),syn45[1:].max()]
plt.subplots(figsize=(12,2)); plt.plot(amplmin_in, 'r'); plt.plot(amplmax_in, 'b');
plt.axis('on'); plt.show()

-------------------------------------------------

### Hubungan Velocity-Pressure secara umum

Hubungan pressure-velocity sangat bersifat empiris dan harus selalu dikalibrasi dengan data lokal. Tidak banyak pengembangan hubungan Vel-Press yang dapat ditemui pada publikasi, pengantar pembahasan hubungan ini secara baik dapat ditemui pada buku Vernik, Seismic Petrophysics in Quantitative Interpertation, SEG, 2016

<br>Beberapa hubungan yang dibahas dalam buku tersebut, dan dapat didiskusikan lebih mendalam misalnya:<br>

Eberhart-Phillips (1989) berdasarkan multivariate Han, et.al.(1986):
<br>
<font size=4>$V_p = 5.77-6.94\phi-1.73V_{cl}^{\frac{1}{2}}+0.446[P_{eff}-exp(16.7.P_{eff})]$</font>
<br><br> mean effective stress dalam kBar<br>
<font size=4>$V_s = 3.70-4.94\phi-1.57V_{cl}^{\frac{1}{2}}+0.361[P_{eff}-exp(16.7.P_{eff})]$</font>
<br>


Vernik menurunkan hubungan empiris dengan shale velocity, perubahan velocity akibat variasi vertical effective stress untuk bedding-normal yang konsisten dengan shale compaction behaviour, diekspresikan sbb:
<font size=4>$V_p(0^o) = V_{pm}(1-\phi_c.exp(-\frac{\sigma_v}{C_m}))^{(d-fV_{cl})}$</font>
<br>
<font size=4>$V_{pm} = a-bv_{cl}+cv_{cl}^2$</font>
<br><br>
a=5.69km/s, b=3.56, c=1.42, d=2.302, f=0.646, $C_m$=27.5MPa(4000psi), $\phi_c$=40%

In [ ]:
initial=[3.08, 2.25, 1.72]
vp1=3.08; vs1=2.25; rho1=1.72
#-----------Eberhart--------
peff = np.linspace(.05,.20,10)
phi0=0.21; vcl0=0.23
vp20=[]; vs20=[]
for i in range(len(peff)):  
    vp0=5.77-6.94*phi0-1.73*(vcl0**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp20.append(vp0) 
    vs0=3.70-4.94*phi0-1.57*(vcl0**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs20.append(vs0) 
vp21=[]; vs21=[]
phi1=0.98*phi0
for i in range(len(peff)):  
    vp1=5.77-6.94*phi1-1.73*(vcl0**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp21.append(vp1) 
    vs1=3.70-4.94*phi1-1.57*(vcl0**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs21.append(vs1) 
vp22=[]; vs22=[]
phi2=1.02*phi0
for i in range(len(peff)):  
    vp2=5.77-6.94*phi2-1.73*(vcl0**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp22.append(vp2) 
    vs2=3.70-4.94*phi2-1.57*(vcl0**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs22.append(vs2) 
vp23=[]; vs23=[]
vcl1=0.85*vcl0
for i in range(len(peff)):  
    vp3=5.77-6.94*phi0-1.73*(vcl1**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp23.append(vp3) 
    vs3=3.70-4.94*phi0-1.57*(vcl1**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs23.append(vs3) 
vp24=[]; vs24=[]
vcl2=0.95*vcl0
for i in range(len(peff)):  
    vp4=5.77-6.94*phi0-1.73*(vcl2**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp24.append(vp4) 
    vs4=3.70-4.94*phi0-1.57*(vcl2**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs24.append(vs4)
vp25=[]; vs25=[]
vcl3=1.05*vcl0
for i in range(len(peff)):  
    vp5=5.77-6.94*phi0-1.73*(vcl3**0.5)+0.446*(peff[i]-(np.exp(-16.7*peff[i])))
    vp25.append(vp5) 
    vs5=3.70-4.94*phi0-1.57*(vcl3**0.5)+0.361*(peff[i]-(np.exp(-16.7*peff[i])))
    vs25.append(vs5)    
f, axs=plt.subplots(ncols=1, figsize=(6,6), sharey=True)
plt.plot(vp20,peff,'ko', mew=6, zorder=5); 
plt.plot(vp20,peff,'C0', lw=6, alpha=0.6, zorder=1)
plt.plot(vp21,peff,'C0', lw=6, alpha=.2)
plt.plot(vp22,peff,'C0', lw=6, alpha=1, label='Phi')
plt.plot(vp23,peff,'C1',lw=6, linestyle="--", alpha=0.2, label='Clay')
plt.plot(vp24,peff,'C1', lw=6, linestyle="--", alpha=0.6)
plt.plot(vp25,peff,'C1', lw=6, linestyle="--", alpha=1)
plt.xlabel('Vpep in km/s', size=16); plt.ylabel('Peff in kBar', size=16)
plt.grid(linestyle=":", color='gray'); plt.legend(frameon=False,loc="upper left", fontsize=12)
plt.show()

increasing effective pressure bisa karena meningkatnya overurden atau pore pressure mengecil (depletion). Injection menyebabkan pore pressure meningkat atau effective pressure berkurang

<br> jika diambil salah satu model di atas misalnya, at: vcl=0.21, phi=0.23, Initial Peff tidak diketahui tetapi lebih kecil dari 6 MPa menurut hubungan di atas. Jika Peff increase menjadi misalnya 0.10 kBar atau 10MPa, maka elastic logs model menjadi Vp=3.44, Vs=1.88, rho=1.63*Vp^0.3=2.36 (Gardner)

In [ ]:
peff[3],vp20[3],vs20[3]

In [ ]:
isi_10MPa=np.array(([3.30, 2.42, 1.85],[3.44, 1.88, 2.36],[3.48, 2.48, 1.85]))

In [ ]:
exec(open('f09displallstacks_10Mpa.py').read())

In [ ]:
amplmin_in=[syn0[:-1].min(),syn5[:-1].min(),syn10[:-1].min(),syn15[:-1].min(),syn20[:-1].min(),syn25[:-1].min(),
         syn30[:-1].min(),syn35[:-1].min(),syn40[:-1].min(),syn45[:-1].min()]
amplmax_in=[syn0[1:].max(),syn5[1:].max(),syn10[1:].max(),syn15[1:].max(),syn20[1:].max(),syn25[1:].max(),
         syn30[1:].max(),syn35[1:].max(),syn40[1:].max(),syn45[1:].max()]
plt.subplots(figsize=(12,2)); plt.plot(amplmin_in, 'r'); plt.plot(amplmax_in, 'b');
plt.axis('on'); plt.show()

<h1><center>-Akhir Program Latihan-9b-<center></h1>